In [1]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

import talib

In [2]:
filepath = r"C:\Users\fleut\ForexAI\data\EURUSD.txt"

peak_width=3

In [3]:
df = pd.read_csv(filepath,
                     sep=',',
                     names=['Ticker','Date','Time', 'Open', 'High', 'Low', 'Close', 'Volume'],
                     skiprows=1)

df.set_index(['Date', 'Time'], drop=False)
df = df.tail(500000)
df

,Ticker,Date,Time,Open,High,Low,Close,Volume
6881454,EURUSD,20200623,90700,1.1291,1.1291,1.1289,1.1289,4
6881455,EURUSD,20200623,90800,1.1289,1.1291,1.1289,1.1290,4
6881456,EURUSD,20200623,90900,1.1289,1.1291,1.1289,1.1291,4
6881457,EURUSD,20200623,91000,1.1292,1.1292,1.1290,1.1291,4
6881458,EURUSD,20200623,91100,1.1290,1.1291,1.1290,1.1290,4
...,...,...,...,...,...,...,...,...
7381449,EURUSD,20211031,235500,1.1557,1.1557,1.1557,1.1557,4
7381450,EURUSD,20211031,235600,1.1557,1.1557,1.1557,1.1557,4
7381451,EURUSD,20211031,235700,1.1557,1.1557,1.1557,1.1557,4
7381452,EURUSD,20211031,235800,1.1557,1.1557,1.1557,1.1557,4


In [4]:
peaks,_ = find_peaks(df.High,width=peak_width)
valley,_ = find_peaks(-1*df.Low,width=peak_width)
lows=df.iloc[valley].Low
highs=df.iloc[peaks].High

In [5]:
hh = highs > highs.shift(1)
df['highest_high_cnt']=0
df['highest_high_len']=0
df['highest_high_ang']=0
hh_cnt = hh.cumsum()-hh.cumsum().where(~hh).ffill().fillna(0).astype(int)
hh_from = np.arange(len(hh_cnt)) - hh_cnt
hh_high = highs - highs.values[hh_from]
df.iloc[peaks,df.columns.get_loc('highest_high_cnt')] = hh_cnt
df.iloc[peaks,df.columns.get_loc('highest_high_len')] = peaks - peaks[hh_from]
df.iloc[peaks,df.columns.get_loc('highest_high_ang')] = hh_high / (peaks - peaks[hh_from])

In [6]:
lh = highs < highs.shift(1)
df['lowest_high_cnt']=0
df['lowest_high_len']=0
df['lowest_high_ang']=0
lh_cnt = lh.cumsum()-lh.cumsum().where(~lh).ffill().fillna(0).astype(int)
lh_from = np.arange(len(lh_cnt)) - lh_cnt
lh_high = highs - highs.values[lh_from]
df.iloc[peaks,df.columns.get_loc('lowest_high_cnt')] = lh_cnt
df.iloc[peaks,df.columns.get_loc('lowest_high_len')] = peaks - peaks[lh_from]
df.iloc[peaks,df.columns.get_loc('lowest_high_ang')] = lh_high / (peaks - peaks[lh_from])

In [7]:
hl = lows > lows.shift(1)
df['highest_low_cnt']=0
df['highest_low_len']=0
df['highest_low_ang']=0
hl_cnt = hl.cumsum()-hl.cumsum().where(~hl).ffill().fillna(0).astype(int)
hl_from = np.arange(len(hl_cnt)) - hl_cnt
hl_high = lows - lows.values[hl_from]
df.iloc[valley,df.columns.get_loc('highest_low_cnt')] = hl_cnt
df.iloc[valley,df.columns.get_loc('highest_low_len')] = valley - valley[hl_from]
df.iloc[valley,df.columns.get_loc('highest_low_ang')] = hl_high / (valley - valley[hl_from])

In [8]:
ll = lows < lows.shift(1)
df['lowest_low_cnt']=0
df['lowest_low_len']=0
df['lowest_low_ang']=0
ll_cnt = ll.cumsum()-ll.cumsum().where(~ll).ffill().fillna(0).astype(int)
ll_from = np.arange(len(ll_cnt)) - ll_cnt
ll_high = lows - lows.values[ll_from]
df.iloc[valley,df.columns.get_loc('lowest_low_cnt')] = ll_cnt
df.iloc[valley,df.columns.get_loc('lowest_low_len')] = valley - valley[ll_from]
df.iloc[valley,df.columns.get_loc('lowest_low_ang')] = ll_high / (valley - valley[ll_from])

In [9]:
ord=2
mask = (df['lowest_high_cnt']>=ord) | (df['lowest_low_cnt']>=ord) | (df['highest_high_cnt']>=ord) | (df['highest_low_cnt']>=ord)
df[mask]

,Ticker,Date,Time,Open,High,Low,Close,Volume,highest_high_cnt,highest_high_len,highest_high_ang,lowest_high_cnt,lowest_high_len,lowest_high_ang,highest_low_cnt,highest_low_len,highest_low_ang,lowest_low_cnt,lowest_low_len,lowest_low_ang
6881514,EURUSD,20200623,100700,1.1283,1.1283,1.1283,1.1283,4,0,0,0.000000,0,0,0.000000,0,0,NaN,2,26,-0.000015
6881553,EURUSD,20200623,104600,1.1284,1.1284,1.1282,1.1283,4,0,0,0.000000,0,0,0.000000,0,0,NaN,2,16,-0.000044
6881579,EURUSD,20200623,111200,1.1287,1.1287,1.1285,1.1285,4,0,0,NaN,2,52,-0.000017,0,0,0.000000,0,0,0.000000
6881617,EURUSD,20200623,115000,1.1297,1.1297,1.1295,1.1296,4,0,0,0.000000,0,0,0.000000,2,64,0.000020,0,0,NaN
6881648,EURUSD,20200623,122100,1.1324,1.1324,1.1321,1.1321,4,2,69,0.000054,0,0,NaN,0,0,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381321,EURUSD,20211029,184700,1.1555,1.1555,1.1555,1.1555,4,0,0,0.000000,0,0,0.000000,0,0,NaN,2,25,-0.000016
7381347,EURUSD,20211031,221300,1.1558,1.1558,1.1558,1.1558,4,0,0,0.000000,0,0,0.000000,2,26,0.000012,0,0,NaN
7381358,EURUSD,20211031,222400,1.1559,1.1559,1.1559,1.1559,4,0,0,0.000000,0,0,0.000000,3,37,0.000011,0,0,NaN
7381435,EURUSD,20211031,234100,1.1559,1.1559,1.1559,1.1559,4,0,0,0.000000,0,0,0.000000,2,54,0.000004,0,0,NaN
